In [ ]:
import pandas as pd
import re
import collections

In [ ]:
menCsv = pd.DataFrame.from_csv('nordstromMen.csv')
womenCsv = pd.DataFrame.from_csv('nordstromWomen.csv')

In [ ]:
class Props:
    def __init__(self):
        self.pieces = self.pieces()
        self.fit = self.fit()
        self.pattern = self.pattern()
        self.texture = self.texture()
        
    def pieces(self):
        return(
            [
            'Shirt','Polo','Shorts','Jacket','Jeans','Pants',
            'Trunks','Hoodie','Sweater','Sweatshirt','Trousers',
            'Pullover','Jersey','Top','Coat','Tank',
            'Jogger','Suit','Blazer','Vest','Chinos'
            ]
        )
    
    def fits(self):
        return(
            [
                'Trim','Slim','Regular','Straight','Stretch',
                'Modern','Tailored','Slub','Relaxed','Skinny',
                'Slouchy'
            ]
        )
        
    def patterns(self):
        return(
            ['Stripe','Plaid','Graphic','Colorblock','Floral',
             'Houndstooth','Indigo','Contrast']
        )
    
    def textures(self):
        return(
            [
                'Nylon','Textured','Cotton','Wool','Linen',
                'Woven','Silk','Twill','Knit','Microfiber',
                'Wrinkle','Jacquard','Mesh','Leather','Gingham',
                'Cashmere','Herringbone','Houndstooth','Fleece'
            ]
        )
    

In [ ]:
class dfExtract(Props):
    
    def __init__(self,df):
        self.df = df
        self.__itemFreqTable = self.__itemFreqTable()
        self.ItemFreqCnt = self.__itemFreqCount()
        self.FitFreqCnt = self.__fitFreqCount()
        self.PatternFreqCnt = self.__patternFreqCount()
        self.TextureFreqCnt = self.__textureFreqCount()
        
    def __itemFreqTable(self):
        itemNames = self.df['ItemName']
        terms = (' '.join(itemNames.tolist())).split(' ')
        
        freqTable = collections.Counter()
        for term in terms:
            freqTable[term] += 1
            
        return(freqTable)
    
    def __itemFreqCount(self):
    #returns the counts of ONLY the selected items
        itemCounts = []
        for item in self.pieces():
            itemCounts.append(self.__itemFreqTable.get(item))
        return(itemCounts)
    
    def __fitFreqCount(self):
    #returns the counts of ONLY the selected fits
        fitCounts = []
        for fit in self.fits():
            fitCounts.append(self.__itemFreqTable.get(fit))
        return(fitCounts)
    
    def __textureFreqCount(self):
    #returns the counts of ONLY the selected textures
        textureCounts = []
        for texture in self.textures():
            textureCounts.append(self.__itemFreqTable.get(texture))
        return(textureCounts)
    
    def __patternFreqCount(self):
    #returns the counts of ONLY the selected patterns
        patternCounts = []
        for pattern in self.patterns():
            patternCounts.append(self.__itemFreqTable.get(pattern))
        return(patternCounts)
    

In [ ]:
class descriptiveStats:
    def __init__(self,df):
        self.data = dfExtract(df)
        self.tFits = self.__totalFits()
        self.tTextures = self.__totalTextures()
        self.tItems = self.__totalItems()
        self.tPatterns = self.__totalPatterns()
    
        self.summary = self.summary()
        self.uniqueSummary = self.uniqueSummary()
        
    def __superSum(self,array):
        #Temporary solution
        total = 0
        for i in array:
            try:
                total += i
            except:
                next
        return(total) 
    
    
    def __totalFits(self):
        return(self.__superSum(self.data.FitFreqCnt))
        
    def __totalTextures(self):
        return(self.__superSum(self.data.TextureFreqCnt))
    
            
    def __totalItems(self):
        return(self.__superSum(self.data.ItemFreqCnt))
    
    def __totalPatterns(self):
        return(self.__superSum(self.data.PatternFreqCnt))
    
    def summary(self):
        return(
            {
                'Fits':self.tFits,
                'Textures':self.tTextures,
                'Items':self.tItems,
                'Patterns':self.tPatterns
            }
        )
    
    def uniqueSummary(self):
        return(
            {
                'Fits':len(self.data.FitFreqCnt),
                'Textures':len(self.data.TextureFreqCnt),
                'Items':len(self.data.ItemFreqCnt),
                'Patterns':len(self.data.PatternFreqCnt)
            }
        )
        

In [ ]:
class plotting:
    pass


In [ ]:
#womenData = descriptiveStats(womenCsv)
#womenData.summary